In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import numpy as np
import re
import matplotlib.colors as mcolors
plt.style.use('default')
import pandas as pd
from sklearn.decomposition import NMF, PCA
from sklearn.cluster import KMeans
from sklearn.manifold import Isomap, TSNE
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed, Embedding, GlobalAveragePooling1D, Flatten, SimpleRNN, GRU, Dropout, LSTM
from tensorflow.keras.utils import plot_model
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import requests
stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
stopwords = set(stopwords_list.decode().splitlines()) 
import os


### **DISCLAIMER**
#### A ideia para essa APS era utilizar a série de livros Harry Potter como dataset, sendo que cada página representa um documento, e os documentos são classificados como positivos ou negativos, dependendo da presença ou não de personagens antagonistas.
#### A partir desses dados seriam gerados entidades nomeadas, e 2 redes neurais seriam treinadas para gerar novos documentos, uma para os de classificação positiva, e uma para os de negativa.
#### Foi possível realizar a parte teórica, mas por diversas limitações de processamento e dados as redes não produziram resultados muito satisfatórios.

In [ ]:
#LOADING DATA

with open("Book1.txt", encoding="utf8") as f:
    Book1 = f.readlines()
with open("Book2.txt", encoding="utf8") as f:
    Book2 = f.readlines()
# with open("Book3.txt", encoding="utf8") as f:
#     Book3 = f.readlines()
# with open("Book4.txt", encoding="utf8") as f:
#     Book4 = f.readlines()
# with open("Book5.txt", encoding="utf8") as f:
#     Book5 = f.readlines()
# with open("Book6.txt", encoding="utf8") as f:
#     Book6 = f.readlines()
# with open("Book7.txt", encoding="utf8") as f:
#     Book7 = f.readlines()
# Books = [Book1, Book2, Book3, Book4, Book5, Book6, Book7]
Books = [Book1, Book2]

           

In [ ]:
#DATA CLEANING
AllBooksText = []
BookTitles = ['Harry Potter and the Philosophers Stone - J.K. Rowling', 'Harry Potter and the Order of the Phoenix - J.K. Rowling', 'Harry Potter and the Half Blood Prince - J.K. Rowling', 'Harry Potter and the Deathly Hallows - J.K. Rowling']
for b in Books:
    b  = ''.join(map(str, b))
    b =  b.replace('\n', '')
    for bt in BookTitles:
        b = b.replace(bt, '')
    bPages = b.split('Page |')
    AllBooksText.append(bPages)



In [ ]:
AllBooksText = [item for sublist in AllBooksText for item in sublist]
AllBooksTextClean = []

In [ ]:
for p in AllBooksText:
    for bt in BookTitles:
        pclean = p.replace(bt, '')
    AllBooksTextClean.append(pclean)
    


In [ ]:
#POS TAGGING
pos_tagged = []


for doc in AllBooksTextClean:
    try:
        doc = re.sub(r'[^A-Za-z0-9 ]+', '', doc)
        wt = nltk.word_tokenize(doc)
        pos_tagged.append(nltk.pos_tag(wt, tagset = "universal"))
    except:
        pass

In [ ]:
#SENTIMENT CLASSIFICATION AND CREATING NAMED ENTITIES
evilCharacters = ['Voldemort', 'Snape', 'Dursley', 'Malfoy', 'Dudley', 'Tom']
pattern = 'NamedEntity: {<NOUN><NOUN>}'
cp = nltk.RegexpParser(pattern)
NamedEntities = []
testList = []
cbad = 0
cgood = 0
bad = False
i = 0
finalList = []
for p in pos_tagged:
    testList = []
    for x in p:
        for eC in evilCharacters:
            if eC in x:
                bad = True
    cs = cp.parse(p)
    for p in cs:
        xlist = []
        try:
            if p.label() == "NamedEntity":
                xlist.append(' '.join([i[0] for i in p.leaves()]))
                xlist.append(AllBooksTextClean[i])
                if bad:
                    xlist.append("EVIL")
                else:
                    xlist.append("GOOD")
        except AttributeError:
            continue
        testList.append(xlist)
    bad = False
    finalList.append(testList)
    i += 1
            
    



In [ ]:
entities = []
texts = []
sentiments = []
for i in finalList:
    for j in i:
        entities.append(j[0])
        texts.append(j[1])
        sentiments.append(j[2])


In [ ]:
dictionary = {"entities":entities, "texts": texts, "sentiments": sentiments}

In [ ]:
df = pd.DataFrame(dictionary)
positives = df[df["sentiments"] == "GOOD"]
negatives = df[df["sentiments"] == "EVIL"]

In [ ]:
positives.head()

In [ ]:
p_entities = positives["entities"]
p_labels = positives["texts"]
n_entities = negatives["entities"]
n_labels = negatives["texts"]

In [ ]:
#TOKENIZING, PADDING AND ENCODING
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokens_entities = Tokenizer(num_words=128)
tokenizer = Tokenizer(num_words=128)
tokenizer.fit_on_texts(positives["texts"])
padded_seq = pad_sequences(tokenizer.texts_to_sequences(positives["texts"]), maxlen=128)
tokens_entities.fit_on_texts(positives["entities"])
X = pad_sequences(tokens_entities.texts_to_sequences(positives["entities"]), maxlen=128)
y_ = np.array(padded_seq)
y_ = y_.reshape(y_.shape[0]*y_.shape[1],1)
onehot_encoder = OneHotEncoder(sparse=False)
y = onehot_encoder.fit_transform(y_)
y = y.reshape(padded_seq.shape[0], 128, X.shape[1])
print(f'{X.shape=} {y.shape=}')


In [ ]:
tokenizer2 = Tokenizer(num_words=128)
tokenizer2.fit_on_texts(negatives["texts"])
padded_seq = pad_sequences(tokenizer2.texts_to_sequences(negatives["texts"]), maxlen=128)
tokens_entities.fit_on_texts(negatives["entities"])
X2 = pad_sequences(tokens_entities.texts_to_sequences(negatives["entities"]), maxlen=128)
y_2 = np.array(padded_seq)
y_2 = y_2.reshape(y_2.shape[0]*y_2.shape[1],1)
onehot_encoder = OneHotEncoder(sparse=False)
y2 = onehot_encoder.fit_transform(y_2)
y2 = y2.reshape(padded_seq.shape[0], 128, X.shape[1])
print(f'{X2.shape=} {y2.shape=}')

In [ ]:
#TRAIN/TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2)



In [ ]:
X = X.astype(int)
Y = y.astype(int)
print((X.shape[1], Y.shape[1]))


In [ ]:
X2 = X2.astype(int)
Y2 = y2.astype(int)

In [ ]:
# from keras.models import Model, load_model
# rede_neural_positiva = load_model('redePositiva')
# rede_neural_negativa = load_model('redeNegativa')


In [ ]:
#NEURAL NETWORK STRUCTURE
from tensorflow.keras.utils import plot_model

def rede_neural_geracao(input_dims, dim_out):
        input_layer = Input(shape=(input_dims,))
        x = input_layer
        x = Embedding(150, 150, name='projecao')(x)
        x, state_h, state_c = LSTM(512, return_sequences=True, return_state=True)(x)
        x = LSTM(512, return_sequences=True) (x, initial_state=[state_h, state_c])
        x = Dropout(0.1)(x)
        y = Dense(dim_out, activation='softmax', name='decisor')(x)
        return Model(input_layer, y)
rede_neural = rede_neural_geracao(X.shape[1], Y.shape[1])
rede_neural.compile(optimizer='adam', loss='categorical_crossentropy')
plot_model(rede_neural, show_shapes=True, show_layer_activations=True)

In [ ]:
def positive_prediction(entity: str):
    X = pad_sequences(tokens_entities.texts_to_sequences([entity]), maxlen=128)
    pred = rede_neural_positiva.predict(X)
    return tokenizer.sequences_to_texts(np.argmax(pred, axis=1))

def negative_prediction(entity: str):
    X = pad_sequences(tokens_entities.texts_to_sequences([entity]), maxlen=128)
    pred = rede_neural_negativa.predict(X)
    return tokenizer.sequences_to_texts(np.argmax(pred, axis=1))

In [ ]:
positive_prediction("Harry Potter")

In [ ]:
negative_prediction("Severus Snape Lord Voldemort")